# EDA

## DF laden und Bibliotheken importieren

In [1]:
# Datenverarbeitung
import pandas as pd

# Visualisierung
import plotly_express as px

# für die Dash-App
from dash import Dash
from dash import dcc

from dash import html

from dash.dependencies import Input, Output

# Future Warnings unterdrücken (für bessere Übersicht)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Displaybreite anpassen
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
# pickle Datei einlesen und anzeigen
df = pd.read_pickle('df_kiva_after_preprocessing.pkl')
df

,funded_amount,loan_amount,completion_rate,activity,sector,country,term_in_months,lender_count,funded_amount_per_lender,borrower_count,borrower_genders_updated,repayment_interval
0,300,300,1.0,Fruits & Vegetables,Food,Pakistan,12,12,25.00,1,fe only,irregular
1,575,575,1.0,Rickshaw,Transportation,Pakistan,11,14,41.07,2,fe only,irregular
2,150,150,1.0,Transportation,Transportation,India,43,6,25.00,1,fe only,bullet
3,200,200,1.0,Embroidery,Arts,Pakistan,11,8,25.00,1,fe only,irregular
4,400,400,1.0,Milk Sales,Food,Pakistan,14,16,25.00,1,fe only,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...
671199,0,25,0.0,Livestock,Agriculture,Paraguay,13,0,0.00,1,fe only,monthly
671200,0,25,0.0,Livestock,Agriculture,Paraguay,13,0,0.00,1,fe only,monthly
671201,25,25,1.0,Livestock,Agriculture,Kenya,13,1,25.00,1,fe only,monthly
671203,0,25,0.0,Livestock,Agriculture,Kenya,13,0,0.00,1,fe only,monthly


## Themen identifizieren / Business Frage

Das Ziel ist eine explorative Datenanalyse zu machen, um Insights für unser Unternehmen zu generieren. Wir verdienen dabei unser Geld durch Provisionen pro Projekt, dass auf unserer Plattform landet. Im Endeffekt geht es uns also darum zu analysieren, wie wir mehr Kunden auf unsere Website bekommen und ein Projekt anmelden lassen.


**Thema 1 - Projektverteilung.**<br>
Wovon hängt die Anzahl angemeldeter Projekte ab? Wie können wir dieses Wissen nutzen, um die Gesamtzahl der Projekte zu steigern und unsere Provision zu erhöhen? Dazu starte ich mit der Sektorenverteilung. Welche Sektoren sind wie viele Projekte zugeordnet? Was macht diese Sektoren aus? Wovon sind sie abhängig?<br><br><br>

>Ideensammlung für weitere Analysen:
>- Analyse Geschlechterverteilung -> Abhängigkeiten?
>- Erfüllungsgrad (funded amount / loan amount) -> Abhängigkeiten? -> guter Erfüllungsgrad = gutes Bild nach Außen (lohnt sich da anzufragen als borrower)
>- mehrfach angefraget Projekte? -> mehrfache anfrage lässt ggf. auf Fehler bei Eingabe hindeuten?
>- Höhe des ausgezahlten Betrags in Abhängigkeit von Länder, Geschlecht, ...
>- Höhe Fund <-> Fund pro Kopf Zusammenhang?
>- wie ist Balkendiagramm der Projektanzahl nach Sektoren, wenn man Erfüllungsgrad mit einbezieht?

## Thema 1 - Projektverteilung

### Projekte pro Sektor

Zunächst erstelle ich ein df, dass nach dem Sektor gruppiert ist und die Anzahl der Projekte auflistet.

In [3]:
# df nach sector gruppieren und die Anzahl der Projekte zählen
df_nr_of_projects = df.groupby(by='sector').size().reset_index(name='number_of_projects')

# df nach Anzahl der Projekte sortieren
df_nr_of_projects.sort_values(by='number_of_projects', ascending=False, inplace=True)

# df index zurücksetzen
df_nr_of_projects.reset_index(drop=True, inplace=True)

# df anzeigen
df_nr_of_projects

,sector,number_of_projects
0,Agriculture,179224
1,Food,135749
2,Retail,123758
3,Services,44699
4,Personal Use,36247
5,Housing,33571
6,Clothing,32480
7,Education,30837
8,Transportation,15455
9,Arts,11994


Jetzt erstelle ich ein bar chart auf Basis dieses df. Mit einem bar chart kann man Verhältnisse von mehreren Gruppen gut erkennen.

In [4]:
# bar chart erstellen
bar_sectors = px.bar(
    data_frame=df_nr_of_projects,
    x='sector',
    y='number_of_projects',
    template='simple_white',                           # Template ändern
    color='sector',                                    # Farbe abhängig vom Sektor
    color_discrete_sequence=['lightgray'],             # zunächst jeden Sektor grau einfärben
    color_discrete_map={'Agriculture': 'green',        # dann die ersten zwei grün hervorheben
                        'Food': 'green'},
    labels={'number_of_projects': 'Anzahl Projekte',   # die Beschriftung anpassen
            'sector': 'Sektoren'},
    title='Anzahl der Projekte pro Sektor'             # Titel erstellen
)

# Titel des bar charts anpassen
bar_sectors.update_layout(
    title_x=0.5,
    title_font=dict(size=25),
    legend=dict(font=dict(size= 12))
)

# bar chart anzeigen
bar_sectors.show()

**Deskriptiver Part:**
- die Projekte des Datensatzes sind auf 15 Sektoren aufgeteilt
- das Diagramm zeigt die Anzahl der Projekte für jeden dieser Sektoren
- auf der x-Achse sind die Sektoren aufgelistet
- auf der y-Achse ist die jeweilige Anzahl der Projekte zu sehen
- die Sektoren sind absteigend nach ihrer Projektanzahl sortiert und dargestellt

**Analytischer Part:**<br>
Es fällt auf, dass die größten beiden Sektoren mit der Lebensmittelindustrie in Zusammenhang stehen. Daher sind diese beiden Balken grün eingefärbt. Es scheinen also viele Projekte, die DIREKT mit Lebensmitteln in Verbindung stehen, angemeldet zu werden. Doch was für einen Anteil von der Gesamtzahl der Projekte nehmen diese beiden Sektoren genau ein?

### Anteil lebensmittelbezogener Projekte

Anteile von wenigen Gruppen lassen sich sehr gut mit einem pie chart darstellen. Um diesen zu erstellen und übersichtlich zu halten, werde ich zunächst eine neue Spalte im df erstellen, die die Sektoren in zwei Kategorien aufteilt.

In [5]:
# weitere Spalte mit Kategorie Food Industry / Other abhängig vom Sektor erstellen
df_nr_of_projects['category'] = df_nr_of_projects['sector'].apply(lambda x: 'Food Industry' if x in('Agriculture', 'Food') else 'Other')

# df ausgeben
df_nr_of_projects

,sector,number_of_projects,category
0,Agriculture,179224,Food Industry
1,Food,135749,Food Industry
2,Retail,123758,Other
3,Services,44699,Other
4,Personal Use,36247,Other
5,Housing,33571,Other
6,Clothing,32480,Other
7,Education,30837,Other
8,Transportation,15455,Other
9,Arts,11994,Other


Nachdem ich die beiden Kategorien erstellt haben, setze ich sie in einem pie chart in Beziehung:

In [6]:
# pie chart erstellen
pie_food = px.pie(
    data_frame=df_nr_of_projects,
    names='category',
    values='number_of_projects',
    color='category',                                    # Farbe nach Kategorie vergeben
    color_discrete_map={'Food Industry': 'green',        # Farbgebung beeinflussen
                        'Other': 'lightgray'},
    title='Anteil der Projekte mit Lebensmittelbezug'
)

# Layout anpassen
pie_food.update_layout(
    legend_traceorder="reversed",                        # Legende umdrehen
    legend_title_text='Kategorien',                      # Legendentitel setzen
    title_x=0.47,
    title_font=dict(size=25),
    legend=dict(font=dict(size= 18))
)

# Kuchstückausrichtung anpassen
pie_food.update_traces(                                
    rotation=170                                         # Kuchenstücke drehen
)  

# pie chart anzeigen
pie_food.show()

**Deskripter Part:**
- das Diagramm zeigt den Anteil der Projekte mit Lebensmittelbezug an der Gesamtzahl der Projekte
- die Projekte sind hier in zwei Kategorien eingeteilt:
    - Food Industry: Projekte mit Lebensmittelbezug (Inhalt: 2 Sektoren)
    - Other: alle anderen Projekte (Inhalt: 13 Sektoren)
- die Kategorie Food Industry nimmt etwa 47% Prozent ein und ist grün eingefärbt
- die Kategorie Other nimmt etwa 53% ein und ist hellgrau eingefärbt

**Analytischer Part:**<br>
Die beiden Kategorien beinhalten unterschiedlich viele Sektoren. Daher wäre eigentlich zu erwarten, dass die Kategorie mit deutlich weniger Sektoren auch deutlich weniger Anteil an der Gesamtzahl der Projekte hat. Wie schon im Balkendiagramm abzusehen war und jetzt auch sehr deutlich wird, ist dem nicht so. Die Kategorie Food Industry nimmt fast die Hälfte der Projekte ein.<br>
Doch was macht diese Kategorie bzw. die Projekte in dieser Kategorie so besonders? Kann man Besonderheiten anhand der Geographie erkennen?

### Geographische Verteilung der Kategorie Food Industry

Zunächst muss ich die Kategorien an das Ursprungs-df anspielen.

In [7]:
# weitere Spalte mit Kategorie Food Industry / Other abhängig vom Sektor erstellen
df['category'] = df['sector'].apply(lambda x: 'Food Industry' if x in('Agriculture', 'Food') else 'Other')

# df ausgeben
df

,funded_amount,loan_amount,completion_rate,activity,sector,country,term_in_months,lender_count,funded_amount_per_lender,borrower_count,borrower_genders_updated,repayment_interval,category
0,300,300,1.0,Fruits & Vegetables,Food,Pakistan,12,12,25.00,1,fe only,irregular,Food Industry
1,575,575,1.0,Rickshaw,Transportation,Pakistan,11,14,41.07,2,fe only,irregular,Other
2,150,150,1.0,Transportation,Transportation,India,43,6,25.00,1,fe only,bullet,Other
3,200,200,1.0,Embroidery,Arts,Pakistan,11,8,25.00,1,fe only,irregular,Other
4,400,400,1.0,Milk Sales,Food,Pakistan,14,16,25.00,1,fe only,monthly,Food Industry
...,...,...,...,...,...,...,...,...,...,...,...,...,...
671199,0,25,0.0,Livestock,Agriculture,Paraguay,13,0,0.00,1,fe only,monthly,Food Industry
671200,0,25,0.0,Livestock,Agriculture,Paraguay,13,0,0.00,1,fe only,monthly,Food Industry
671201,25,25,1.0,Livestock,Agriculture,Kenya,13,1,25.00,1,fe only,monthly,Food Industry
671203,0,25,0.0,Livestock,Agriculture,Kenya,13,0,0.00,1,fe only,monthly,Food Industry


Jetzt schränke ich das df auf die Zeilen ein, die Projekte der Kategorie Food Industry beschreiben.

In [8]:
# df einschränken auf Kategorie Food Industry
df_limit = df.loc[df.loc[:, 'category']=='Food Industry', :].copy()

# df ausgeben
df_limit

,funded_amount,loan_amount,completion_rate,activity,sector,country,term_in_months,lender_count,funded_amount_per_lender,borrower_count,borrower_genders_updated,repayment_interval,category
0,300,300,1.0,Fruits & Vegetables,Food,Pakistan,12,12,25.000000,1,fe only,irregular,Food Industry
4,400,400,1.0,Milk Sales,Food,Pakistan,14,16,25.000000,1,fe only,monthly,Food Industry
6,200,200,1.0,Dairy,Agriculture,India,43,8,25.000000,1,fe only,bullet,Food Industry
9,625,625,1.0,Food Production/Sales,Food,Pakistan,11,24,26.040001,1,fe only,irregular,Food Industry
14,225,225,1.0,Poultry,Agriculture,India,43,7,32.139999,1,fe only,bullet,Food Industry
...,...,...,...,...,...,...,...,...,...,...,...,...,...
671199,0,25,0.0,Livestock,Agriculture,Paraguay,13,0,0.000000,1,fe only,monthly,Food Industry
671200,0,25,0.0,Livestock,Agriculture,Paraguay,13,0,0.000000,1,fe only,monthly,Food Industry
671201,25,25,1.0,Livestock,Agriculture,Kenya,13,1,25.000000,1,fe only,monthly,Food Industry
671203,0,25,0.0,Livestock,Agriculture,Kenya,13,0,0.000000,1,fe only,monthly,Food Industry


Jetzt gruppiere ich nach Ländern und Liste die Anzahl der Projekte auf.

In [9]:
# df nach country gruppieren und die Anzahl der Projekte zählen
df_limit_nr_of_project = df_limit.groupby(by='country').size().reset_index(name='number_of_projects')

# df nach Anzahl der Projekte sortieren
df_limit_nr_of_project.sort_values(by='number_of_projects', ascending=False, inplace=True)

# df index zurücksetzen
df_limit_nr_of_project.reset_index(drop=True, inplace=True)

# df anzeigen
df_limit_nr_of_project

,country,number_of_projects
0,Philippines,79446
1,Kenya,47382
2,El Salvador,22021
3,Cambodia,11744
4,Peru,11624
...,...,...
82,Guam,0
83,Virgin Islands,0
84,Lesotho,0
85,Lao People's Democratic Republic,0


Da die Anzahl der Projekte für die Länder extrem weit auseinander liegt und dadurch Probleme bei der Visualisierung entstehen, werde ich die Daten kategorisieren. Außerdem entferne ich die Zeilen mit 0 Projekten mit Lebensmittelbezug.

In [10]:
# labels definieren
my_label = ['<=1000', '>1000 <=5000', '>5000 <=12000', '~22000', '~47000', '~79000']                                    

# Kategoriegrenzen definieren
my_bins = [0, 1000, 5000, 12000, 23000, 48000, 80000]                       

# Kategorie-Spalte erstellen
df_limit_nr_of_project.loc[:, 'number_of_projects_cat'] = pd.cut(
    x=df_limit_nr_of_project.loc[:, 'number_of_projects'],
    bins=my_bins,
    labels=my_label
)

# NULL-Werte entfernen (Länder mit 0 Projekten mit Lebensmittelbezug)
df_limit_nr_of_project.dropna(subset=['number_of_projects_cat'], inplace=True)

# df anzeigen
df_limit_nr_of_project

,country,number_of_projects,number_of_projects_cat
0,Philippines,79446,~79000
1,Kenya,47382,~47000
2,El Salvador,22021,~22000
3,Cambodia,11744,>5000 <=12000
4,Peru,11624,>5000 <=12000
...,...,...,...
73,Puerto Rico,31,<=1000
74,Somalia,16,<=1000
75,Saint Vincent and the Grenadines,8,<=1000
76,Chile,2,<=1000


Auf Basis dieses df erstelle ich jetzt eine choropleth map. Diese zeigt farblich die Anzahl der Projekte pro Land auf einer Weltkarte.

In [11]:
# choropleth erstellen
geo_food = px.choropleth(
    df_limit_nr_of_project,
    locations="country",
    locationmode='country names',                           # so kann die map die Länderschreibweise aus dem df interpretieren
    color='number_of_projects_cat',                         # Farbe orientiert sich an Anzahl-Kategorie-Spalte
    template='simple_white',                                # Template wieder simple white
    title='Projektanzahl mit Lebensmittelbezug weltweit',
    width=1500,                                             # Breite manuell anpassen
    height=1500,                                            # Höhe manuell anpassen
    projection='mercator',                                  # mit dieser Projektion sieht man am meisten
    scope='world',                                          # dargestellt wird die gesamte Welt
    color_discrete_map={                                    # den einzelnen Anzahl-Kategorien eine bestimmte Farbe zuweisen (grüner Verlauf)
        '<=1000': '#f0f7da',
        '>1000 <=5000': '#c9df8a',
        '>5000 <=12000': '#77ab59',
        '~22000': '#ff5252',
        '~47000': '#ff0000',
        '~79000': '#a70000'
    },
    labels={'number_of_projects_cat': 'Projektanzahl'}
)

# Layout anpassen
geo_food.update_layout(
    title_x = 0.5,                                          # Titel in die Mitte schieben
    title_font=dict(size=25),                               # Schriftgröße des Titels anpassen
    legend=dict(                                            # Legende anpassen (Schriftgröße und Position)
        font=dict(size= 18),
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.98
    )
)

# choropleth anzeigen
geo_food.show()

**Deskriptiver Part:**
- das Diagramm zeigt eine Weltkarte (es kann rein und rausgezoomt werden)
- die Länder wurden abhängig von ihrer Anzahl angemeldeter Projekte in Kategorien eingeteilt - diese sind in der Legende zu sehen
    - es gibt drei Kategorien, die in einer grünen Farbpalette eingefärbt wurden (hier sind jeweils mehrere Länder in jeder Kategorie - Intensität mit Projektanzahl steigend)
    - außerdem gibt es drei Extremwerte, die sich deutlich von den Gruppen aber auch von den anderen Extremwerten unterscheiden
        - diese sind rot eingefärbt (Intensität mit Projektanzahl steigend)
- es fällt auf, dass vor allem Amerika, Südostafrika, der nahe Osten und Asien eingefärbt sind
- Extremwerte liegen auf den Phillipinen, in Kenia und in El Salvador

**Analytischer Part:**<br>
Es fallen zwei Dinge auf:
1. Die drei Länder mit den höchsten Werten (rot) heben sich DEUTLICH von den Gruppen (grün) aber auch voneinander ab. Was macht sie aus?
2. Manche Bereiche (Europa, Ozeanien, Russland, Nordafrika, Kanada und anderen einzelne Länder) haben gar keine lebensmittelbezogenen Projekte angemeldet. Was unterscheidet diese Bereiche von den eingefärbten? Haben sie keinen Zugang zu unserer Website oder können keine Projekte anmelden? Brauchen sie keine Projekte, weil sie von ihrem jeweiligen Staat unterstützt werden?

Was könnte man darauf aufbauend machen?
- unsere anderen Projekte (einzeln und zusammen) auf einer solchen Karte mappen
    - so kann man Zugung zur Website und Projektanmeldung untersuchen
    - außerdem kann man so Interesse der Länder an Projekten aus verschiedenen Sektoren abbilden
- das BIP der Länder mit einbeziehen
    - Lebensmittelindustrie spiegelt eines unserer Grundbedürfnisse wieder. Vielleicht melden viele Länder solche lebensmittelbezogenen Projekte an, weil sie dieses Bedürfnis so besser erfüllt bekommen?
- die Politik der Ländern mit einbeziehen
    - staatliche Subventionen in der Lebensmittelindustrie?

# Dashboards

## Projektanzahl mit Lebensmittelbezug mit auswählbarem Sichtbereich

In [12]:
# 1. App erstellen

choropleth_scope = Dash(__name__)



# 2. Layout festlegen

choropleth_scope.layout = html.Div(
    # Liste mit allen Core Components und HTML Components
    children=[
        # Überschrift
        html.H1(
            children='Projektanzahl mit Lebensmittelbezug mit auswählbarem Sichtbereich',
            style={'text-align': 'center'}
        ),
        
        # Dropdown
        dcc.Dropdown(
            id='dd_scope', # unique identifier für den callback
            options=['world', 'europe', 'asia', 'africa', 'north america', 'south america'],
            placeholder='Sichtbereich wählen:',
            value='world',
            style={'width': '40%'} # Breite der Dropdown Liste anpassen
        ),
        
        # Grafik
        dcc.Graph(
            id='choro_nr_of_projects_per_scope', # unique identifier für den callback
            figure={} # hier wird return value vom callback reingeschrieben
        )
    ]
)



# 3. Callback erstellen 

@choropleth_scope.callback(
    Output(component_id='choro_nr_of_projects_per_scope', component_property='figure'),
    Input(component_id='dd_scope', component_property='value')
)
def update_choro(selected_scope):
    
    # df anreichern und einschränken
    
    # weitere Spalte mit Kategorie hinzufügen
    df['category'] = df['sector'].apply(lambda x: 'Food Industry' if x in('Agriculture', 'Food') else 'Other')
    
    # df einschränken auf Kategorie Food Industry
    df_limit = df.loc[df.loc[:, 'category']=='Food Industry', :].copy()
    
    
    # df gruppieren und zählen
    
    # df nach country gruppieren und die Anzahl der Projekte zählen
    df_limit_nr_of_project = df_limit.groupby(by='country').size().reset_index(name='number_of_projects')

    # df nach Anzahl der Projekte sortieren
    df_limit_nr_of_project.sort_values(by='number_of_projects', ascending=False, inplace=True)

    # df index zurücksetzen
    df_limit_nr_of_project.reset_index(drop=True, inplace=True)
    
    
    # Kategorien definieren
    
    # labels definieren
    my_label = ['<=1000', '>1000 <=5000', '>5000 <=12000', '~22000', '~47000', '~79000']                                    

    # Kategoriegrenzen definieren
    my_bins = [0, 1000, 5000, 12000, 23000, 48000, 80000]                       

    # Kategorie-Spalte erstellen
    df_limit_nr_of_project.loc[:, 'number_of_projects_cat'] = pd.cut(
        x=df_limit_nr_of_project.loc[:, 'number_of_projects'],
        bins=my_bins,
        labels=my_label
    )

    # NULL-Werte entfernen (Länder mit 0 Projekten mit Lebensmittelbezug)
    df_limit_nr_of_project.dropna(subset=['number_of_projects_cat'], inplace=True)
    
    scope_=selected_scope
    
    # Plotten
    
    # choropleth erstellen
    geo_food_industry = px.choropleth(
        df_limit_nr_of_project,
        locations="country",
        locationmode='country names',                           # so kann er die Länderschreibweise aus dem df interpretieren
        color='number_of_projects_cat',                         # Farbe orientiert sich an Anzahl-Kategorie-Spalte
        template='simple_white',                                # Template wieder simple white
        width=1500,                                             # Breite manuell anpassen
        height=1500,                                            # Höhe manuell anpassen
        projection='mercator',                                  # mit dieser Projektion sieht man am meisten
        scope=scope_,                                           # Sichtbereich
        color_discrete_map={                                    # den einzelnen Anzahl-Kategorien eine bestimmte Farbe zuweisen (grüner Verlauf)
            '<=1000': '#f0f7da',
            '>1000 <=5000': '#c9df8a',
            '>5000 <=12000': '#77ab59',
            '~22000': '#ff5252',
            '~47000': '#ff0000',
            '~79000': '#a70000'
        },
        labels={'number_of_projects_cat': 'Projektanzahl'}
    )
    
    return geo_food_industry



# 4. App starten

choropleth_scope.run(
    port=8891,
    jupyter_mode="tab"
)

Dash app running on http://127.0.0.1:8891/


<IPython.core.display.Javascript object>